In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

# Load breast cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Standardize the data
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize neural network parameters
n_features = X_train.shape[1]
n_hidden1 = 128
n_hidden2 = 128
n_epochs = 1000
learning_rate = 0.01

# Initialize weights and biases
weights = [np.random.randn(n_features, n_hidden1),
           np.random.randn(n_hidden1, n_hidden2),
           np.random.randn(n_hidden2, 1)]
biases = [np.zeros((1, n_hidden1)),
          np.zeros((1, n_hidden2)),
          np.zeros((1, 1))]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def forward(X, weights, biases):
    a1 = X @ weights[0] + biases[0]
    h1 = sigmoid(a1)
    a2 = h1 @ weights[1] + biases[1]
    h2 = sigmoid(a2)
    a3 = h2 @ weights[2] + biases[2]
    output = sigmoid(a3)
    return output, h1, h2

def backpropagation(X, y, weights, biases, a, h1, h2, learning_rate):
    error = a - y
    delta3 = error * a * (1 - a)
    delta_b3 = np.sum(delta3, axis=0, keepdims=True) / X.shape[0]
    delta_w3 = h2.T @ delta3 / X.shape[0]

    delta2 = (delta3 @ weights[2].T) * h2 * (1 - h2)
    delta_b2 = np.sum(delta2, axis=0, keepdims=True) / X.shape[0]
    delta_w2 = h1.T @ delta2 / X.shape[0]

    delta1 = (delta2 @ weights[1].T) * h1 * (1 - h1)
    delta_b1 = np.sum(delta1, axis=0, keepdims=True) / X.shape[0]
    delta_w1 = X.T @ delta1 / X.shape[0]

    # Update weights and biases
    weights[0] -= learning_rate * delta_w1
    biases[0] -= learning_rate * delta_b1
    weights[1] -= learning_rate * delta_w2
    biases[1] -= learning_rate * delta_b2
    weights[2] -= learning_rate * delta_w3
    biases[2] -= learning_rate * delta_b3

    return weights, biases

# Training the neural network
for epoch in range(1, n_epochs + 1):
    a, h1, h2 = forward(X_train, weights, biases)
    weights, biases = backpropagation(X_train, y_train.reshape(-1, 1), weights, biases, a, h1, h2, learning_rate)

# Evaluate the performance of the trained neural network on the test set
y_pred, _, _ = forward(X_test, weights, biases)
y_pred_class = np.round(y_pred)
accuracy = np.mean(y_pred_class.flatten() == y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 90.35%
